This writeup documents one way to solve a particular mystery location challenge. There are three pieces of infomation given:

- the photo below, which is taken at the location
- the location is somewhere in southern Finland
- the photo was taken in December 2016

All tools, datasets, and techniques are allowed.

![a photo of the mystery location](./location.jpg)




# Prerequsites

This analysis was originally done in a Jupyter Notebook using a python3 kernel. Install the required python packages using `pip`:

```
python3 -m pip install geopandas folium overpass geojson
```

`geopandas` provides geospatial data processing datastrucrues and algoritms, `folium` provieds flippy map visualization, and `overpass` will help with collecting data from OpenStreetMap.

# Bounding the search area

In order to limit the size of the search area, let's assume that the location we are looking for is in the coastal areas of south-west finland.

The data will be collected form using overpass, so we need a bounding box that overpass understands. 

1. go to <http://overpass-turbo.org> 
2. in the map view, use the *manually set bbox* button.
3. set the bounding area
4. copy the coordinates from: *export > map > map view > bounds*.
    It should look somehting like `59.80202, 21.18439, 60.82751, 23.59863`

In [1]:
# save the bounding for use in overpass queries
bbox = '59.80202, 21.18439, 60.82751, 23.59863'
# the same box will later be used by folium, but in a different format
bounds = [[59.80202, 23.59863],[60.82751, 21.18439]]

tourism_query = 'node["tourism"="information"]({bbox})'.format(bbox = bbox)
#aktia_query = 'node["name"="Aktia"]({bbox})'.format(bbox = bbox)
aktia_query = 'node["amenity"="bank"]({bbox})'.format(bbox = bbox)

print(tourism_query)
print(aktia_query)

node["tourism"="information"](59.80202, 21.18439, 60.82751, 23.59863)
node["amenity"="bank"](59.80202, 21.18439, 60.82751, 23.59863)


With the queries built, the next step is to call Overpass Turbo to get the data. It's possible to use any overpass server, but there is no reason not to use the default one for the purpous of this analysis.

In [2]:
import overpass

api = overpass.API()
tourisms = api.Get(tourism_query)
aktias = api.Get(aktia_query)


In [3]:

# in order to make sure the data is not lost, wirte it ot a file
open("tourisms.geojson", "w").write(str(tourisms)); 
open("aktias.geojson",   "w").write(str(aktias));


We can inspect the value data returned by the query by by leaving the variable name on the last line of a code cell. The returned data is in Geojson, which is great, but in order to better understand it we should look at it on a map. It is possible to just print the raw geojson, bit it's not very intuivitve.

In [4]:
# print(tourisms)

Folium is python package that allows quickly creating interactive maps using OpensStreetMap tiles.

Folium has support for a number of common tilesets without needing to know the TMS url for the them. In this case we use cartodb's positron tiles. The map is bounded to the query area.

In [5]:
import folium
import geojson

# in order to not duplicate code
# we'll loop over the two datasets and 
# create two maps
maps = []

for dataset in [tourisms, aktias]:
    mapview = folium.Map(tiles='cartodbpositron')
    mapview.fit_bounds(bounds)
    points = folium.features.GeoJson(geojson.dumps(dataset))
    mapview.add_children(points)
    maps.append(mapview)


## Locations of the tourism information nodes

In [6]:
maps[0]


## Locations of the Aktia bank nodes

In [7]:
maps[1]

# Determining co-location

In order to determine the locations where there is a banks and a tourist info close to each other, we'll create a 50 meter buffer around each node in the datasets, and the look for intersections in the buffered datasets. This means any place with a tourist info and bank within 100 meters of each other will show up in the end result.

In [8]:
import geopandas
import matplotlib.pyplot as plt

infos = geopandas.read_file("tourisms.geojson");
banks = geopandas.read_file("aktias.geojson");
print(infos.crs)
print(banks.crs)



{'init': 'epsg:4326'}
{'init': 'epsg:4326'}


geopandas has a `dataset.to_crs()` function, which will be used to reporoject the geometries into a projection which has a metric northing/easting crs definition.

Once the dataset is reporjected, we create a 50 meter buffer around all the nodes in the dataset using the `GeoSeries.buffer()` function. Once buffered each dataset will consist of multiple polygons. In order to more easily work with the polygons, they are merged into a single multipolygon using `GeoSeries.unary_union`

The intersection is calculated once the datasets are buffered and made into multipolygons.

In [9]:
utm_zones_north_proj4 = "utm +zone=33 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
datasets = [infos, banks]
multipolygons = []
for index, dataset in enumerate(datasets):
    dataset = dataset.to_crs({'proj': utm_zones_north_proj4})
    dataset.geometry = dataset.buffer(50)
    dataset = dataset.to_crs({'init': 'epsg:4326'})
    multipolygons.append(dataset.unary_union)

intersection = multipolygons[0].intersection(multipolygons[1])

In order view the result, we'll create another folium map with the intersection polygons, and markers at the polygon centroids since the polygons will be relatively small and difficult to see when zoomed out.

In [10]:
import folium
import geojson
# show the map
mapview = folium.Map(tiles='cartodbpositron')
mapview.fit_bounds(bounds)
for location in intersection: 
    outline  = folium.features.GeoJson(geojson.dumps(location))
    centroid = folium.features.GeoJson(geojson.dumps(location.centroid))
    mapview.add_children(outline)
    mapview.add_children(centroid)
mapview

# Result verification

Now that there is only a manageable number of candidate locations left, we can use google street view to visit them in order to determine which one is the mystery location. In order to make the process streamlined, we can use a little bit of python to open multiple streetviews, one for each location, and embedding them in this notebook.

The `IPython.display.HTML` function allows us to create and embed html into notebooks. We loop over each polygon in the intersection multipolygon, and request a google street view at the centroid of each of those them.

The embedded streetviews are not directly from the google maps API, but using the [ctrlq map embed utility](https://ctrlq.org/maps/embed/), which gives a convenient way to get streetviews from arbitrary coordinates.

In [11]:
from IPython.display import HTML

html_code = ''
for location in intersection:
    html_code = html_code + """
        <iframe style="width:100%;height:250px;"
        frameborder="0" scrolling="no" marginheight="0" marginwidth="0"
        src="http://my.ctrlq.org/maps/#street|0.01|0|0|{y_coord}|{x_coord}">
        </iframe>
    """.format(
        x_coord = location.centroid.x,
        y_coord = location.centroid.y
    )

HTML(html_code)



Looking at the road, it's clear from the cobble stone street that the location in Ekenäs is what we are looking for. The builing with the tourist information sign and the Aktia bank are also visible when panning around the view.

